In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image as kp_image
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.applications import VGG19
from tensorflow.keras import backend as K

# Load and preprocess the image
def load_and_process_image(img_path):
    img = kp_image.load_img(img_path, target_size=(224, 224))
    img = kp_image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = tf.keras.applications.vgg19.preprocess_input(img)
    return img

# De-process the image to get it back to displayable format
def deprocess_image(img):
    img = img.numpy()
    img = img.reshape((224, 224, 3))
    img = img[:, :, ::-1]  # Convert back to BGR
    img += np.array([103.939, 116.779, 123.68])  # Add the RGB mean
    img = np.clip(img, 0, 255).astype('uint8')
    return img

# Load pre-trained VGG19 model + higher level layers (we don't need the final layers)
def get_model():
    vgg = VGG19(weights='imagenet', include_top=False)
    vgg.trainable = False
    return vgg

# Define the content and style layers for the model
def get_feature_extractor(model, content_layers, style_layers):
    outputs = [model.get_layer(name).output for name in content_layers + style_layers]
    return tf.keras.Model(inputs=model.input, outputs=outputs)

# Content loss function
def content_loss(base_content, target_content):
    return tf.reduce_mean(tf.square(base_content - target_content))

# Style loss function
def style_loss(base_style, target_style):
    return tf.reduce_mean(tf.square(base_style - target_style))

# Total loss function (content + style)
def total_loss(content_weight, style_weight, content_features, style_features, generated_image, model):
    # Get model outputs for generated image
    model_outputs = feature_extractor(generated_image)
    
    # Extract content and style features for the generated image
    content_features_gen = model_outputs[:len(content_layers)]
    style_features_gen = model_outputs[len(content_layers):]
    
    # Compute content and style loss
    c_loss = content_weight * content_loss(content_features, content_features_gen)
    s_loss = style_weight * style_loss(style_features, style_features_gen)
    
    # Compute total loss
    total_loss_value = c_loss + s_loss
    return total_loss_value, c_loss, s_loss

# Compute the gradients for optimizing the generated image
@tf.function()
def compute_gradients(generated_image, content_features, style_features):
    with tf.GradientTape() as tape:
        tape.watch(generated_image)
        total_loss_value, c_loss, s_loss = total_loss(content_weight, style_weight, content_features, style_features, generated_image, model)
    grads = tape.gradient(total_loss_value, generated_image)
    return grads, total_loss_value

# Hyperparameters
content_weight = 1e3
style_weight = 1e-2
content_layers = ['block5_conv2']  # Content layer to use
style_layers = ['block1_conv1', 'block2_conv1', 'block3_conv1', 'block4_conv1', 'block5_conv1']  # Style layers to use

# Load content and style images
content_image_path = 'content.jpg'  # Path to the content image
style_image_path = 'style.jpg'      # Path to the style image

content_image = load_and_process_image(content_image_path)
style_image = load_and_process_image(style_image_path)

# Create the model
model = get_model()

# Get feature extractor
feature_extractor = get_feature_extractor(model, content_layers, style_layers)

# Get content and style features
content_output = feature_extractor(content_image)[0]  # content output is at index 0
style_output = feature_extractor(style_image)[1:]  # style outputs are at indices 1, 2, ...

# Initialize generated image (start with the content image)
generated_image = tf.Variable(content_image)

# Optimization loop
optimizer = tf.optimizers.Adam(learning_rate=0.02)

iterations = 1000
for i in range(iterations):
    grads, total_loss_value = compute_gradients(generated_image, content_output, style_output)
    optimizer.apply_gradients([(grads, generated_image)])

    if i % 100 == 0:
        print(f"Iteration {i}, Loss: {total_loss_value.numpy()}")
        img = deprocess_image(generated_image)
        plt.imshow(img)
        plt.show()

# Final output after optimization
final_img = deprocess_image(generated_image)
plt.imshow(final_img)
plt.title("Generated Image")
plt.show()

2025-04-24 16:08:41.040990: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-24 16:08:41.071356: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-24 16:08:41.071660: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-24 16:08:41.646004: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


FileNotFoundError: [Errno 2] No such file or directory: 'content.jpg'